In [6]:
from sys import path
path.append("./deep-belief-network-master/")
from dbn import SupervisedDBNRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
import os

In [9]:
def RMSE(data1,data2):
    return np.sqrt(np.mean((data1-data2)*(data1-data2)))
def MAE(data1,data2):
    return np.mean(np.abs(data1-data2))
def BIAS(data1,data2):
    return np.mean(data1-data2)
def UBRMSE(data1,data2):
    rmse=RMSE(data1,data2)
    bias=BIAS(data1,data2)
    return np.sqrt(rmse*rmse-bias*bias)
def PCC(data1,data2):
    return pearsonr(data1,data2)[0]

In [11]:
def DBNTrain(datapath,hidden,rbm_lr,lr,rbm_epoch,epoch,batchsize):
    data=pd.read_csv(datapath)
    predictors=data[["Ele","Albedo","SSM","UW","LST","NDVI","DOY","AT"]].values
    outputvar=data["Mean"].values
    # Splitting data
    X_train, X_test, Y_train, Y_test = train_test_split(predictors, outputvar, test_size=0.2)
    # Data scaling
    min_max_scaler = MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    # Training                                               
    regressor = SupervisedDBNRegression(hidden_layers_structure=hidden,
                                        learning_rate_rbm=rbm_lr,
                                        learning_rate=lr,
                                        n_epochs_rbm=rbm_epoch,
                                        n_iter_backprop=epoch,
                                        batch_size=batchsize,
                                        activation_function='relu')
    regressor.fit(X_train, Y_train)
    X_test = min_max_scaler.transform(X_test)
    Y_pred = regressor.predict(X_test)
    print(PCC(Y_pred[:,0],Y_test))
    return regressor